# Test Selection Analysis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import distance

In [7]:
def load_data(train, test, fname) :
    fpath = "trained_models/resnet18_{}_piecewise_eps8_bs256_maxlr0.1_ls0.3_BNeval/eval/{}/{}.txt".format(train, test, fname)
    if train == "all" or train == "combine_autopgd_pixelattack_spatialtransformation_squareattack" or train == "combine_newtonfool_pixelattack_spatialtransformation_squareattack":
            fpath = "trained_models/resnet18_{}_piecewise_eps8_bs128_maxlr0.1_ls0.3_BNeval/eval/{}/{}.txt".format(train, test, fname)
    return np.loadtxt(fpath).astype(np.int)

train = "autoattack"
test = "pgd"

y_original = load_data(train, test, "Y_original")
y_original_pred = load_data(train, test, "Y_original_pred")
y_adv = load_data(train, test, "Y_adv")
y_adv_pred = load_data(train, test, "Y_adv_pred")

In [8]:
def get_robust_data(train, test):
    y_adv = load_data(train, test, "Y_adv")
    y_adv_pred = load_data(train, test, "Y_adv_pred")
    return y_adv, y_adv_pred

train = "pgd"
test = "autoattack"
y_adv, y_adv_pred = get_robust_data(train, test)

print("Y_adv({},{}): {}".format(train, test, y_adv))
print("Y_adv_pred({},{}): {}".format(train, test, y_adv_pred))

Y_adv(pgd,autoattack): [3 8 8 ... 5 1 7]
Y_adv_pred(pgd,autoattack): [3 8 0 ... 5 0 7]


In [9]:
train = "fgsm"
test = "autopgd"
y_adv, y_adv_pred = get_robust_data(train, test)

print("Y_adv({},{}): {}".format(train, test, y_adv))
print("Y_adv_pred({},{}): {}".format(train, test, y_adv_pred))

Y_adv(fgsm,autopgd): [3 8 8 ... 5 1 7]
Y_adv_pred(fgsm,autopgd): [3 8 8 ... 5 1 7]


In [10]:
attacks = ["autoattack", "autopgd", "bim", "cw", "fgsm", "pgd", "deepfool", "newtonfool", "squareattack", "pixelattack", "spatialtransformation"]

In [13]:
train = attacks.copy()
train.append("all")
# train.append("combine_autopgd_pixelattack_spatialtransformation_squareattack")
train.append("combine_newtonfool_pixelattack_spatialtransformation_squareattack")

test = attacks.copy()
test.append("all")

In [14]:
predictions = {}
labels = {}

for a1 in train :
    p = np.array([])
    l = np.array([])
    for a2 in attacks :
        y, y_pred = get_robust_data(a1, a2)
        p = np.concatenate((p, y_pred))
        l = np.concatenate((l, y))
    predictions[a1] = {}
    labels[a1] = {}
    predictions[a1]["all"] = p
    labels[a1]["all"] = l

for a1 in train :
    for a2 in attacks :
        y, y_pred = get_robust_data(a1, a2)
        predictions[a1][a2] = y_pred
        labels[a1][a2] = y

In [15]:
# predictions

In [16]:
# labels

In [17]:
accs = {}
for tr in train :
    accs[tr] = {}
    for ts in test :
        accs[tr][ts] = (labels[tr][ts] == predictions[tr][ts]).sum().item() / len(labels[tr][ts])

    
accs = pd.DataFrame(data=accs).T
accs

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all
autoattack,0.7300,0.7310,0.7296,0.7422,0.7109,0.7279,0.7278,0.7432,0.7329,0.1097,0.3272,0.637491
autopgd,0.7254,0.7262,0.7227,0.7351,0.6999,0.7239,0.7215,0.7343,0.7322,0.1337,0.3646,0.638136
bim,0.7212,0.7221,0.7222,0.7263,0.7045,0.7207,0.7148,0.7259,0.7229,0.1265,0.3426,0.631791
cw,0.7253,0.7209,0.7202,0.7406,0.7021,0.7203,0.7290,0.7392,0.7353,0.1166,0.3432,0.635700
fgsm,0.7121,0.7108,0.7106,0.7169,0.7033,0.7101,0.7058,0.7172,0.7081,0.1160,0.3468,0.623427
pgd,0.7290,0.7263,0.7249,0.7377,0.7095,0.7246,0.7288,0.7369,0.7315,0.1408,0.3151,0.636827
deepfool,0.7138,0.7125,0.7115,0.7327,0.6995,0.7130,0.7274,0.7317,0.7233,0.0974,0.3551,0.628900
newtonfool,0.7248,0.7257,0.7227,0.7384,0.7061,0.7201,0.7277,0.7388,0.7354,0.1332,0.3588,0.639245
squareattack,0.6564,0.6556,0.6553,0.7085,0.6288,0.6519,0.6617,0.7088,0.7629,0.1508,0.3551,0.599618
pixelattack,0.1001,0.1001,0.1002,0.1002,0.1001,0.1001,0.1002,0.1002,0.1002,0.7021,0.1000,0.154864


In [18]:
sorted_acc = accs.sort_values(by="all", ascending=False)
sorted_acc

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all
all,0.7312,0.7275,0.7264,0.7441,0.7074,0.7269,0.7342,0.7450,0.7499,0.1859,0.4590,0.657955
combine_newtonfool_pixelattack_spatialtransformation_squareattack,0.6972,0.6928,0.6928,0.7366,0.6624,0.6875,0.7014,0.7344,0.7602,0.1835,0.5613,0.646373
newtonfool,0.7248,0.7257,0.7227,0.7384,0.7061,0.7201,0.7277,0.7388,0.7354,0.1332,0.3588,0.639245
autopgd,0.7254,0.7262,0.7227,0.7351,0.6999,0.7239,0.7215,0.7343,0.7322,0.1337,0.3646,0.638136
autoattack,0.7300,0.7310,0.7296,0.7422,0.7109,0.7279,0.7278,0.7432,0.7329,0.1097,0.3272,0.637491
pgd,0.7290,0.7263,0.7249,0.7377,0.7095,0.7246,0.7288,0.7369,0.7315,0.1408,0.3151,0.636827
cw,0.7253,0.7209,0.7202,0.7406,0.7021,0.7203,0.7290,0.7392,0.7353,0.1166,0.3432,0.635700
bim,0.7212,0.7221,0.7222,0.7263,0.7045,0.7207,0.7148,0.7259,0.7229,0.1265,0.3426,0.631791
deepfool,0.7138,0.7125,0.7115,0.7327,0.6995,0.7130,0.7274,0.7317,0.7233,0.0974,0.3551,0.628900
fgsm,0.7121,0.7108,0.7106,0.7169,0.7033,0.7101,0.7058,0.7172,0.7081,0.1160,0.3468,0.623427


## Time Execution Measurement

In [27]:
import datetime

def get_adv_train_time_execution(train) :
    fpath = "trained_models/{}/output.log".format(train)
    fpath = "trained_models/resnet18_{}_piecewise_eps8_bs256_maxlr0.1_ls0.3_BNeval/output.log".format(train)
    if train == "all" or train == "combine_autopgd_pixelattack_spatialtransformation_squareattack" or train == "combine_newtonfool_pixelattack_spatialtransformation_squareattack":
            fpath = "trained_models/resnet18_{}_piecewise_eps8_bs128_maxlr0.1_ls0.3_BNeval/output.log".format(train)

    
    file = open(fpath)
    lines = file.readlines()
    file.close()
    
    start_time = None
    end_time = None
    
    datefmt='%Y/%m/%d %H:%M:%S'
    
    
    for l in lines :
        token = l.split("-")
        date_str = token[0][1:-2]
        date_time = datetime.datetime.strptime(date_str, datefmt)
        if token[1][1:11] == "Namespace(" :
            start_time = date_time
        else :
            end_time = date_time
    
    if end_time > start_time :
        return end_time-start_time
    raise ValueError("Duration not found")
    
_train = "autoattack"
_train = "all"
get_adv_train_time_execution(_train) + get_adv_train_time_execution("fgsm") 

datetime.timedelta(seconds=54108)

In [28]:
durations = []
for tr in train :
    if tr[:7] == "combine" :
        duration = None
        for i in range(len(attacks)) :
            a = attacks[i]
            if i == 0 :
                duration = get_adv_train_time_execution(a)
            else :
                duration += get_adv_train_time_execution(a)
        duration+= get_adv_train_time_execution(tr)
        durations.append(duration)
    else :
        durations.append(get_adv_train_time_execution(tr))

accs["duration"] = durations

In [29]:
accs

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all,duration
autoattack,0.7300,0.7310,0.7296,0.7422,0.7109,0.7279,0.7278,0.7432,0.7329,0.1097,0.3272,0.637491,0 days 02:05:26
autopgd,0.7254,0.7262,0.7227,0.7351,0.6999,0.7239,0.7215,0.7343,0.7322,0.1337,0.3646,0.638136,0 days 02:04:21
bim,0.7212,0.7221,0.7222,0.7263,0.7045,0.7207,0.7148,0.7259,0.7229,0.1265,0.3426,0.631791,0 days 02:03:13
cw,0.7253,0.7209,0.7202,0.7406,0.7021,0.7203,0.7290,0.7392,0.7353,0.1166,0.3432,0.635700,0 days 02:04:57
fgsm,0.7121,0.7108,0.7106,0.7169,0.7033,0.7101,0.7058,0.7172,0.7081,0.1160,0.3468,0.623427,0 days 02:02:49
pgd,0.7290,0.7263,0.7249,0.7377,0.7095,0.7246,0.7288,0.7369,0.7315,0.1408,0.3151,0.636827,0 days 03:03:10
deepfool,0.7138,0.7125,0.7115,0.7327,0.6995,0.7130,0.7274,0.7317,0.7233,0.0974,0.3551,0.628900,0 days 02:03:55
newtonfool,0.7248,0.7257,0.7227,0.7384,0.7061,0.7201,0.7277,0.7388,0.7354,0.1332,0.3588,0.639245,0 days 01:06:00
squareattack,0.6564,0.6556,0.6553,0.7085,0.6288,0.6519,0.6617,0.7088,0.7629,0.1508,0.3551,0.599618,0 days 02:44:20
pixelattack,0.1001,0.1001,0.1002,0.1002,0.1001,0.1001,0.1002,0.1002,0.1002,0.7021,0.1000,0.154864,0 days 03:01:23


In [30]:
sorted_acc = accs.sort_values(by="all", ascending=False)
sorted_acc

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all,duration
all,0.7312,0.7275,0.7264,0.7441,0.7074,0.7269,0.7342,0.7450,0.7499,0.1859,0.4590,0.657955,0 days 12:58:59
combine_newtonfool_pixelattack_spatialtransformation_squareattack,0.6972,0.6928,0.6928,0.7366,0.6624,0.6875,0.7014,0.7344,0.7602,0.1835,0.5613,0.646373,1 days 05:41:09
newtonfool,0.7248,0.7257,0.7227,0.7384,0.7061,0.7201,0.7277,0.7388,0.7354,0.1332,0.3588,0.639245,0 days 01:06:00
autopgd,0.7254,0.7262,0.7227,0.7351,0.6999,0.7239,0.7215,0.7343,0.7322,0.1337,0.3646,0.638136,0 days 02:04:21
autoattack,0.7300,0.7310,0.7296,0.7422,0.7109,0.7279,0.7278,0.7432,0.7329,0.1097,0.3272,0.637491,0 days 02:05:26
pgd,0.7290,0.7263,0.7249,0.7377,0.7095,0.7246,0.7288,0.7369,0.7315,0.1408,0.3151,0.636827,0 days 03:03:10
cw,0.7253,0.7209,0.7202,0.7406,0.7021,0.7203,0.7290,0.7392,0.7353,0.1166,0.3432,0.635700,0 days 02:04:57
bim,0.7212,0.7221,0.7222,0.7263,0.7045,0.7207,0.7148,0.7259,0.7229,0.1265,0.3426,0.631791,0 days 02:03:13
deepfool,0.7138,0.7125,0.7115,0.7327,0.6995,0.7130,0.7274,0.7317,0.7233,0.0974,0.3551,0.628900,0 days 02:03:55
fgsm,0.7121,0.7108,0.7106,0.7169,0.7033,0.7101,0.7058,0.7172,0.7081,0.1160,0.3468,0.623427,0 days 02:02:49


In [32]:
selected_sorted_acc = sorted_acc[:][["all", "duration"]]
selected_sorted_acc

,all,duration
all,0.657955,0 days 12:58:59
combine_newtonfool_pixelattack_spatialtransformation_squareattack,0.646373,1 days 05:41:09
newtonfool,0.639245,0 days 01:06:00
autopgd,0.638136,0 days 02:04:21
autoattack,0.637491,0 days 02:05:26
pgd,0.636827,0 days 03:03:10
cw,0.635700,0 days 02:04:57
bim,0.631791,0 days 02:03:13
deepfool,0.628900,0 days 02:03:55
fgsm,0.623427,0 days 02:02:49
